<a href="https://colab.research.google.com/github/GowshikPrabhu/flutter-flowerclassifer/blob/master/FlowerClassfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os

In [4]:
# Get the dataset file
URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(fname='flower_photos.tgz', origin=URL, extract=True, cache_subdir='/content')

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [6]:
IMAGE_SIZE = 244
BATCH_SIZE = 64
# data generator from image
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
# Create train and test data
train_generator = datagen.flow_from_directory(
    base_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset='validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [7]:
# Create a label files
labels = "\n".join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)

In [9]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
# Create a pretrained model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    weights='imagenet'
)

9412608/9406464 [==============================] - 0s 0us/step


In [12]:
base_model.trainable = False

In [14]:
# Create a multilayer sequential model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5,activation='softmax')
])

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
epochs = 10

history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
46/46 [==============================] - 165s 4s/step - loss: 0.7108 - accuracy: 0.7686 - val_loss: 0.4395 - val_accuracy: 0.8427
Epoch 2/10
46/46 [==============================] - 161s 4s/step - loss: 0.2606 - accuracy: 0.9098 - val_loss: 0.4483 - val_accuracy: 0.8618
Epoch 3/10
46/46 [==============================] - 162s 4s/step - loss: 0.1580 - accuracy: 0.9493 - val_loss: 0.4397 - val_accuracy: 0.8536
Epoch 4/10
46/46 [==============================] - 163s 4s/step - loss: 0.1028 - accuracy: 0.9677 - val_loss: 0.4613 - val_accuracy: 0.8673
Epoch 5/10
46/46 [==============================] - 159s 3s/step - loss: 0.0677 - accuracy: 0.9809 - val_loss: 0.4584 - val_accuracy: 0.8687
Epoch 6/10
46/46 [==============================] - 159s 3s/step - loss: 0.0351 - accuracy: 0.9922 - val_loss: 0.4766 - val_accuracy: 0.8673
Epoch 7/10
46/46 [==============================] - 163s 4s/step - loss: 0.0286 - accuracy: 0.9939 - val_loss: 0.5701 - val_accuracy: 0.8591
Epoch 8/10
46

In [17]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [18]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>